<img src="../../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

## Sequential Organ Failure Assessment (SOFA)
The sequential organ failure assessment score (SOFA score), originally published as as the Sepsis-related Organ Failure Assessment score ([Vincent et al., 1996](http://link.springer.com/10.1007/BF01709751)), is a disease severity score designed to track the severity of critical ilness throughout the ICU stay. In contrast to APACHE (II/IV), which only calculates a score for the first 24 hours, it can be used sequentially for every following day. The code performs some data cleanup and calculates the SOFA score for the first 24 hours of ICU admission for all patients in the database.

**Note**: Requires creating the [dictionaries](../../dictionaries/create_dictionaries.ipynb) before running this notebook.

## Imports

In [5]:
%matplotlib inline
import amsterdamumcdb
import psycopg2
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl

import io
from IPython.display import display, HTML, Markdown

In [2]:
#matplotlib settings for image size
#needs to be in a different cell from %matplotlib inline
plt.style.use('seaborn-darkgrid')
plt.rcParams["figure.dpi"] = 288
plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["font.size"] = 12

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

In [3]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../config.ini'):
    config.read('../config.ini')
else:
    config.read('../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)

cursor = con.cursor()
cursor.execute('SET SCHEMA \'amsterdamumcdb\''); #set search_path to amsterdamumcdb schema

In [4]:
if not os.path.exists('feature'):
    os.mkdir('feature')

In [20]:
csv_file = 'feature/vaso.csv'
sql = """
        SELECT 
        n.admissionid,
        n.ordercategory,
        (n.start - a.admittedat)/(1000*60) AS start,
        (n.stop - a.admittedat)/(1000*60) AS stop,
        a.admittedat/(1000*60) AS time_s,
        a.dischargedat/(1000*60) AS time_d,
        a.lengthofstay,
        n.duration,
        a.weightgroup,
        n.itemid,
        n.dose,
        n.rate,
        n.doseunit,
        n.fluidin,
        n.rateunit
    FROM drugitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (    
            6818, -- Adrenaline (Epinefrine)
            7135, -- Isoprenaline (Isuprel)
            7178, -- Dobutamine (Dobutrex)
            7179, -- Dopamine (Inotropin)
            7196, -- Enoximon (Perfan)
            7229, -- Noradrenaline (Norepinefrine)
            12467, -- Terlipressine (Glypressin)
            13490, -- Methyleenblauw IV (Methylthionide cloride)
            19929 -- Fenylefrine
    )
    ORDER BY admissionid, start, stop

    """
data = pd.read_sql(sql, con)
data.to_csv(csv_file, index=False)

In [6]:
csv_file = 'feature/fluidin.csv'
sql = """
        SELECT 
        n.admissionid,
        n.ordercategoryid,
        n.duration,
        (n.start - a.admittedat)/(1000*60) AS start,
        (n.stop - a.admittedat)/(1000*60) AS stop,
        a.admittedat/(1000*60) AS time_s,
        a.dischargedat/(1000*60) AS time_d,
        a.lengthofstay,
        n.ordercategory,
        a.weightgroup,
        n.itemid,
        n.dose,
        n.rate,
        n.doseunit,
        n.fluidin,
        n.rateunit
    FROM drugitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE fluidin > 0 AND ordercategoryid in (65, 61, 17, 55, 15, 24, 23, 25, 67, 27, 26, 39) 
    AND itemid NOT IN (    
            6818, -- Adrenaline (Epinefrine)
            7135, -- Isoprenaline (Isuprel)
            7178, -- Dobutamine (Dobutrex)
            7179, -- Dopamine (Inotropin)
            7196, -- Enoximon (Perfan)
            7229, -- Noradrenaline (Norepinefrine)
            12467, -- Terlipressine (Glypressin)
            13490, -- Methyleenblauw IV (Methylthionide cloride)
            19929 -- Fenylefrine
    )
    -- WHERE fluidin > 0 AND ordercategoryid in (65, 61, 17, 55)
    ORDER BY admissionid, start, stop

    """
data = pd.read_sql(sql, con)
data.to_csv(csv_file, index=False)

In [7]:
# demographics
sql = """
SELECT admissionid,gender,agegroup,weightgroup
FROM admissions
        ORDER BY admissionid
"""
demo = pd.read_sql(sql, con)
demo.to_csv('feature/demo.csv', index=False, columns=['admissionid','time','itemid','value'])

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['time', 'itemid', 'value'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [8]:
#get Glasgow Coma Scale-score
sql = """
WITH gcs_components AS (
    SELECT 
        eyes.admissionid,
        --eyes.itemid,
        --eyes.item,
        --eyes.value,
        --eyes.valueid,
        CASE eyes.itemid
            WHEN 6732 THEN 5 - eyes.valueid     --Actief openen van de ogen
            WHEN 13077 THEN eyes.valueid        --A_Eye
            WHEN 14470 THEN eyes.valueid - 4    --RA_Eye   
            WHEN 16628 THEN eyes.valueid - 4    --MCA_Eye
            WHEN 19635 THEN eyes.valueid - 4    --E_EMV_NICE_24uur
            WHEN 19638 THEN eyes.valueid - 8    --E_EMV_NICE_Opname
        END AS eyes_score,
        --motor.value,
        --motor.valueid,
        CASE motor.itemid
            WHEN 6734 THEN 7 - motor.valueid    --Beste motore reactie van de armen
            WHEN 13072 THEN motor.valueid       --A_Motoriek
            WHEN 14476 THEN motor.valueid - 6   --RA_Motoriek
            WHEN 16634 THEN motor.valueid - 6   --MCA_Motoriek
            WHEN 19636 THEN motor.valueid - 6   --M_EMV_NICE_24uur
            WHEN 19639 THEN motor.valueid - 12  --M_EMV_NICE_Opname
        END AS motor_score,
        --verbal.value,
        --verbal.valueid,
        CASE verbal.itemid
            WHEN 6735 THEN 6 - verbal.valueid   --Beste verbale reactie
            WHEN 13066 THEN verbal.valueid      --A_Verbal
            WHEN 14482 THEN verbal.valueid - 5  --RA_Verbal
            WHEN 16640 THEN verbal.valueid - 5  --MCA_Verbal
            WHEN 19637 THEN verbal.valueid - 9 --V_EMV_NICE_24uur
            WHEN 19640 THEN verbal.valueid - 15 --V_EMV_NICE_Opname
        END AS verbal_score,
        eyes.registeredby,
        (eyes.measuredat - a.admittedat)/(1000*60) AS time       
    FROM listitems eyes
    LEFT JOIN admissions a ON
        eyes.admissionid = a.admissionid
    LEFT JOIN listitems motor ON
        eyes.admissionid = motor.admissionid AND
        eyes.measuredat = motor.measuredat AND
        motor.itemid IN (
            6734, --Beste motore reactie van de armen
            13072, --A_Motoriek
            14476, --RA_Motoriek
            16634, --MCA_Motoriek
            19636, --M_EMV_NICE_24uur
            19639 --M_EMV_NICE_Opname   
        )
    LEFT JOIN listitems verbal ON
        eyes.admissionid = verbal.admissionid AND
        eyes.measuredat = verbal.measuredat AND
        verbal.itemid IN (
            6735, --Beste verbale reactie
            13066, --A_Verbal
            14482, --RA_Verbal
            16640, --MCA_Verbal
            19637, --V_EMV_NICE_24uur
            19640 --V_EMV_NICE_Opname
        )
    WHERE eyes.itemid IN (
        6732, --Actief openen van de ogen
        13077, --A_Eye
        14470, --RA_Eye    
        16628, --MCA_Eye
        19635, --E_EMV_NICE_24uur
        19638 --E_EMV_NICE_Opname
        )
    -- measurements within 24 hours of ICU stay:
    AND (eyes.measuredat - a.admittedat) >= 0 
)
SELECT admissionid, time, 0 AS itemid,
        eyes_score + motor_score + (
            CASE 
                WHEN verbal_score < 1 THEN 1
                ELSE verbal_score 
            END
        ) AS value
    FROM gcs_components
        ORDER BY admissionid, time
    
"""
GCS = pd.read_sql(sql, con)
GCS['itemid']=0
GCS.to_csv("feature/GCS.csv", index=False)

In [7]:
'Fio2 pao2 can be found in sofa_respiration.csv'

'Fio2 pao2 can be found in sofa_respiration.csv'

In [5]:
table_ids_list = [
    
    ['SpO2','''
            6709
            '''
    ],
    ['HR','''
            6640
            '''
    ],
    ['SysBP','''
            6641
            '''
    ],
    ['DiaBP','''
            6643
            '''
    ],
    ['MeanBP','''
            6642, --ABP gemiddeld
            6679, --Niet invasieve bloeddruk gemiddeld
            8843 --ABP gemiddeld II
            '''
    ],
    

    ['Sodium', '''
            9924, --Natrium (bloed)
            6840, --Natrium
            9555, --Natrium Astrup
            10284 --Na (onv.ISE) (bloed)
            '''
    ],
    ['WBC_count', '''
            9965, --Leuco's (bloed) 10^9/l
            6779 --Leucocyten 10^9/l
            '''
    ],
    ['Platelets_count', '''
            9964, --Thrombo's (bloed)
            6797, --Thrombocyten
            10409, --Thrombo's citr. bloed (bloed)
            14252 --Thrombo CD61 (bloed)
            '''
    ],
    ['Glucose', '''
            9947, --Glucose (bloed)
            6833, --Glucose Bloed 
            9557 --Glucose Astrup
            '''
    ],
    ['Temp_C','''
            8658, --Temp Bloed
            8659, --Temperatuur Perifeer 2
            8662, --Temperatuur Perifeer 1
            13058, --Temp Rectaal
            13059, --Temp Lies
            13060, --Temp Axillair
            13061, --Temp Oraal
            13062, --Temp Oor
            13063, --Temp Huid
            13952, --Temp Blaas
            16110 --Temp Oesophagus
            '''
    ],
    ['RR','''
            --Evita Parameters
            8873, --Ademfrequentie Evita: measurement by Evita ventilator, most accurate
            --7726, --Ademfrequentie Spontaan: measurement by Evita ventilator, spontaneous breaths/min, distiction not needed for 'actual' respiratory rate
            --9654, --Ademfrequentie Spontaan(2): measurement by 2nd simultaneously used Evita ventilator (very uncommon), spontaneous breaths/min, distiction not needed for 'actual' respiratory rate

            --Servo-i/Servo-u Parameters
            --12283, --Adem Frequentie (Set): setting on Servo-i/Servo-U ventilator, not needed for 'actual' respiratory rate
            --12322, --Adem Frequentie (Set) (2): setting on 2nd simultaneously used Servo-i/Servo-U ventilator (uncommon), not needed for 'actual' respiratory rate
            12266, --Ademfreq.: measurement by Servo-i/Servo-U ventilator, most accurate
            --12348, --Ademfreq.(2): measurement by 2nd simultaneously used Servo-i/Servo-U ventilator (uncommon), no additional information
            --12577 --Ademfreq. Spontaan nieuw --from Servo-i/Servo-U ventilator, spontaneous breaths/min, distiction not needed for 'actual' respiratory rate

            --Patient monitor
            8874 --Ademfrequentie Monitor: measurement by patient monitor using ECG-impedance, less accurate
            '''
    ],
    ['Creatinine','''
            6836, --Kreatinine µmol/l (erroneously documented as µmol)
            9941, --Kreatinine (bloed) µmol/l
            14216 --KREAT enzym. (bloed) µmol/l
            '''
    ],
    ['Potassium','''
            9927, --Kalium (bloed) mmol/l
            9556, --Kalium Astrup mmol/l
            6835, --Kalium mmol/l
            10285 --K (onv.ISE) (bloed) mmol/l
            '''
    ],
    ['Magnesium', '''
            9952, --Magnesium (bloed)
            6839 --Magnesium
            '''
    ],
    ['Albumin', '''
            9937, --Alb.Chem (bloed)
            6801 --Albumine chemisch
            '''
    ],
    ['Arterial_lactate', '''
            10053, --Lactaat (bloed)
            6837, --Laktaat
            9580 --Laktaat Astrup
            '''
    ],
    ['Total_bili', '''
            9945, --Bilirubine (bloed)
            6813 --Bili Totaal
            '''
    ],
    ['Arterial_BE', '''
            9994, --B.E. (bloed)
            6807 --B.E.
            '''
    ],
    ['Hb', '''
            9960, --Hb (bloed)
            6778, --Hemoglobine
            10286, --Hb(v.Bgs) (bloed)
            19703, --Hb poct (bloed)
            9553 --CtHB Astrup
            '''
    ],
    ['Arterial_pH', '''
            14467,
            13080,
            16625,
            12310, --pH (bloed)
            6848 --PH
            '''
    ],
    ['HCO3', '''
            9992, --Act.HCO3 (bloed)
            6810 --HCO3
            '''
    ],
    ['', '''
            '''
    ],
    ['', '''
            '''
    ],
    ['', '''
            '''
    ],
    
]
for table, ids in tqdm(table_ids_list):
    csv_file = "feature/{:s}.csv".format(table)
    if os.path.exists(csv_file) or len(table) == 0:
        continue
    sql = """
        SELECT 
        n.admissionid,
        (n.measuredat - a.admittedat)/(1000*60) AS time,
        n.itemid,
        n.value
    FROM numericitems n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE itemid IN (    {:s}   )
    AND (n.measuredat - a.admittedat) >= 0
    ORDER BY admissionid, time

    """.format(ids)
    data = pd.read_sql(sql, con)
    data.to_csv(csv_file, index=False)

100%|██████████| 24/24 [00:04<00:00,  4.85it/s]


In [18]:
'''


,Chloride, BUN,,Calcium,Ionised_Ca,CO2_mEqL,SGOT,SGPT,,
PTT,PT,INR,
paCO2,
Shock_Index,


bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,
mortality_90d,delay_end_of_record_and_discharge_or_death,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,
Temp_C,FiO2_1,Potassium,Sodium,Chloride,Glucose,BUN,Creatinine,Magnesium,Calcium,Ionised_Ca,CO2_mEqL,
SGOT,SGPT,Total_bili,Albumin,Hb,WBC_count,Platelets_count,PTT,PT,INR,Arterial_pH,paO2,paCO2,Arterial_BE,
Arterial_lactate,HCO3,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,
output_total,output_4hourly,cumulated_balance,SOFA,SIRS


bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,
mortality_90d,delay_end_of_record_and_discharge_or_death,Weight_kg,GCS,HR,SysBP,MeanBP,DiaBP,RR,SpO2,
Temp_C,FiO2_1,Potassium,Sodium,Chloride,Glucose,BUN,Creatinine,Magnesium,Calcium,Ionised_Ca,CO2_mEqL,
SGOT,SGPT,Total_bili,Albumin,Hb,WBC_count,Platelets_count,PTT,PT,INR,Arterial_pH,paO2,paCO2,Arterial_BE,
Arterial_lactate,HCO3,mechvent,Shock_Index,PaO2_FiO2,median_dose_vaso,max_dose_vaso,input_total,input_4hourly,
output_total,output_4hourly,cumulated_balance,SOFA,SIRS

'''

'\n\n,Chloride, BUN,,Calcium,Ionised_Ca,CO2_mEqL,SGOT,SGPT,,\nPTT,PT,INR,Arterial_pH,paCO2,,HCO3,mechvent,\nShock_Index,\n'